# <font color='red' style='font-size: 30px;'> Rede Complexa Baseada em CDR's </font>
<hr style='border: 2px solid red;'>

## <font color = 'black' style='font-size: 26px;'> Imports </font>  
<hr style = 'border: 1.5px solid black;'>

In [3]:
# Lista de Imports
import pandas as pd
import numpy as np
import networkx as nx
import time
from functions import save_with_pickle, load_with_pickle

# Variáveis Globais
EXECUTAR = True
AMOSTRAGEM = True

## <font color = 'black' style='font-size: 26px;'> Importando os Dados </font>  
<hr style = 'border: 1.5px solid black;'>

In [5]:
# Importação dos Dados
dataset = pd.read_csv('../Dados/cdr_processed_Uberlândia.csv', sep=',')

## <font color = 'black' style='font-size: 26px;'> Realizando o Pré-Processamento </font>  
<hr style = 'border: 1.5px solid black;'>

In [7]:
# Se amostragem True, filtra os dados do df conforme os nós salvos anteriormente.  
if AMOSTRAGEM:
    amostragem_nos = load_with_pickle('../Dados/Processados/', 'amostragem_nos')
    dataset = dataset.query("ID_Origem in @amostragem_nos & ID_Destino in @amostragem_nos")

In [8]:
len(amostragem_nos)

127280

In [9]:
# Selecionando os usuários que tenham feito pelo menos 4 ligações para o mesmo destino
df = dataset.groupby(['ID_Origem', 'ID_Destino']).size().reset_index(name='Count')
df = df.query("Count >= 4")
df.head(3)

,ID_Origem,ID_Destino,Count
0,0002448B2A2A138F154E6E90C6D23232,60112502E474CDE92996E61657B0E2E4,16
1,0006E1F1B5CC32F85742C92482A80D25,58D85384ED25AF6E431B0C7E1551F137,4
2,0007220530EE60EFCAEF56FF0B85346C,5FE002659FB06D9E83BD0E22F9F6A7B3,22


In [10]:
agrup_origem = df.groupby('ID_Origem').sum().reset_index()
agrup_origem = agrup_origem.query("Count <= 300")['ID_Origem']

In [11]:
agrup_destino = df.groupby('ID_Destino').sum().reset_index()
agrup_destino = agrup_destino.query("Count <= 300")['ID_Destino']

In [12]:
# Todos os IDs que fizeram ou receberam mais de 6 ligações
nos = np.unique(np.append(agrup_origem, agrup_destino))
save_with_pickle(nos, '../Dados/Processados/', 'nos')
len(nos)

102257

In [13]:
df = df.query("ID_Origem in @nos | ID_Destino in @nos")
df.shape

(152181, 3)

In [14]:
if EXECUTAR:    
    # Instanciando o dicionário
    users = {}
    for ID in df['ID_Origem'].unique():
        users[ID] = {'Ligações': list()}
        
    # Agrupando as ligações em um dicionário    
    for _, row in df.iterrows():
        users[row['ID_Origem']]['Ligações'].append(row['ID_Destino'])
    
    # Salva o dicionário em um objeto pickle
    save_with_pickle(users, '../Dados/Processados/', 'dict_dados')
    
else:
    # Carrega o dicionário em um objeto pickle
    users = load_with_pickle('../Dados/Processados/', 'dict_dados')